In [148]:
from base_dd import BDD_EVENTS
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression

In [149]:
variable_qanti = []
variable_quali = ['NOC']
variable = variable_quali + variable_qanti

#Non eq
#Séléction des va et encodage
df = BDD_EVENTS.loc[:, variable + ['Medal']]
df = pd.get_dummies(df, columns=variable_quali, drop_first=True, dtype=int)
df.loc[df['Medal'].isna(), 'Medal'] = 0
df.loc[df['Medal']!=0, 'Medal'] = 1


#Filtrage
#df = df[BDD_EVENTS['Sport'].isin(['Basketball'])]
df = df.dropna(axis=0, how='any')


X = df.drop(columns='Medal')
Y = df['Medal']
Y = np.round(Y).astype(int)


YG = Y[df['Medal']==1]
XG = X[df['Medal']==1]


#Eq
dfa = df.copy()
dfa_1 = dfa[dfa["Medal"] == 1]
dfa_0 = dfa[dfa["Medal"] == 0]
dfa_0_sampled = dfa_0.sample(n=len(dfa_1), random_state=42)
dfa = pd.concat([dfa_1, dfa_0_sampled], axis=0).sample(frac=1, random_state=42).reset_index(drop=True)


Xa = dfa.drop(columns='Medal')
Ya = dfa['Medal']
Ya = np.round(Ya).astype(int)


YGa = Ya[dfa['Medal']==1]
XGa = Xa[dfa['Medal']==1]


In [150]:
model = LogisticRegression(solver='newton-cg')
model.fit(X, Y)
print(model.coef_)
print(model.score(X, Y))
print(model.score(XG, YG))

[[-0.57495692 -0.92192017 -0.13324442 -1.38218468 -1.63956301 -1.25107215
   2.50082958  0.89690794  0.7059208  -0.69134391 -0.63937791  1.73510892
   0.95585472  1.56957524  1.14958131 -0.80905845 -1.21883535  0.24153874
   1.31932941 -0.92876447 -1.19763611 -0.62844121 -1.25511649 -0.96190254
   0.82279949  0.76498201 -1.32372756 -0.6538026   1.33900687 -0.30297078
  -0.2440201   1.06685093 -0.71097017 -0.90092454 -0.87178264  1.47682239
  -1.00552363 -1.12560108 -0.68129304 -0.02591684  1.86548306 -0.63472654
   0.68966935 -1.06298208 -0.67107671 -0.30510133 -0.40513636 -0.38913604
  -0.69524342  1.7043171  -0.26407728  1.6762077  -1.23648844  0.80841528
   1.69388924 -0.16183862 -0.40513636 -0.30678734 -1.12693465 -0.7520834
  -0.22752174 -1.52473223  1.01114914  0.48761854  1.45717328  2.54782004
   0.44125512  1.67603528  1.48022653  1.76016436 -0.5069516  -0.44144614
  -0.81725346  1.70682417 -0.43596017  2.80829483  1.18623883 -0.67107671
   2.03699391  0.59209678  0.8590397   

In [151]:
model = LogisticRegression(solver='newton-cg', class_weight='balanced')
model.fit(X, Y)
print(model.coef_)
print(model.score(X, Y))
print(model.score(XG, YG))


[[-5.83451168e-01 -1.36599486e+00  1.38930009e-01 -1.88417597e+00
  -2.15302567e+00 -1.74158505e+00  2.80043810e+00  1.18332743e+00
   1.00401004e+00 -1.08225271e+00 -1.01516103e+00  2.01965205e+00
   1.24156964e+00  1.86654367e+00  1.44592141e+00 -1.22969153e+00
  -1.38928941e+00  5.67959765e-01  1.60501239e+00 -1.37411908e+00
  -1.36371714e+00 -1.00086578e+00 -1.74603515e+00 -1.41324024e+00
   1.10932555e+00  1.08700825e+00 -1.82102202e+00 -6.86157178e-01
   1.62440993e+00 -1.20759848e-01 -4.43024630e-01  1.35226400e+00
  -1.10725295e+00 -1.34097600e+00 -1.30600142e+00  1.76119607e+00
  -1.46421572e+00 -1.60173194e+00 -1.06937985e+00  2.54503494e-01
   2.15012842e+00 -5.15745748e-01  9.91927736e-01 -1.53050890e+00
  -1.05624528e+00 -2.93042011e-02 -6.92404243e-01 -6.68828911e-01
  -5.53157950e-01  1.99311528e+00 -4.75652998e-01  1.96153467e+00
  -1.41054307e+00  1.09558657e+00  1.97885449e+00 -1.66617443e-02
  -6.92404243e-01 -5.63319568e-02 -1.16016317e+00 -4.88019552e-01
  -1.10748

In [152]:
modela = LogisticRegression(solver='newton-cg')
modela.fit(Xa, Ya)
print(modela.coef_)
print(modela.score(X, Y))
print(modela.score(XG, YG))

[[-5.42477586e-01 -1.07174360e+00 -8.01828362e-02 -1.40915028e+00
  -1.73268972e+00 -1.36678007e+00  2.31799875e+00  8.83104728e-01
   4.72963229e-01 -7.83786281e-01 -4.59020835e-01  1.70743099e+00
   9.20863694e-01  1.34021280e+00  1.38215939e+00 -9.22470585e-01
  -1.27722984e+00  2.64252657e-01  1.38604274e+00 -1.00121155e+00
  -1.39035884e+00 -6.72175675e-01 -1.24798553e+00 -9.62988351e-01
   8.18896476e-01  7.19531366e-01 -1.29796665e+00 -4.44035400e-01
   1.32055652e+00 -2.16189786e-01 -2.66504556e-01  1.05484605e+00
  -6.72175675e-01 -9.22470585e-01 -7.83786281e-01  1.52033065e+00
  -1.07174360e+00 -1.24798553e+00 -8.33277755e-01  8.89128755e-02
   1.88131272e+00 -4.81887666e-01  3.78902815e-01 -9.62988351e-01
  -6.08438744e-01 -2.25186336e-01 -2.66504556e-01 -3.69566284e-01
  -6.00903106e-01  1.69615189e+00 -1.45460373e-01  1.72157075e+00
  -1.37519378e+00  8.57712327e-01  1.67777429e+00 -3.31531568e-02
  -1.45460373e-01 -3.81444270e-01 -1.11529442e+00 -7.28057705e-01
   6.40957

In [159]:
X.columns.get_loc("NOC_GDR")

75

In [160]:
modela.coef_[0][75]

np.float64(2.822074423150098)